## Zigbang 원룸 매물 데이터 수집
- https://www.zigbang.com/

In [9]:
import requests
import pandas as pd
import geohash2

In [10]:
# geohash2 설치
#!pip install geohash2

개발자 도구 > network > fetch/xhr

- 절차(웹 서비스 분석을 통해 절차를 알아내자!)
    - 동이름으로 위도 경도 수집
    - 위도 경도로 geohash(영역) 변환
    - geohash(영역)로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

### 1. 동이름으로 위도 경도 구하기

In [14]:
addr = "강남구 역삼동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

### 2. 위도 경도로 geohash 변환

In [6]:
# install geohash2
# !pip install geohash2

In [15]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm6'

### 3. geohash로 매물 아이디 가져오기

In [16]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [21]:
items = response.json()["items"]
len(items), items[:2]
ids = [item["item_id"] for item in items]
len(ids), ids[:5]

(2344, [32819635, 32838798, 32872608, 32838700, 32767020])

### 4. 매물 아이디로 매물 정보 가져오기

In [23]:
# 1000개 넘어가면 나눠서 수집해야 함
url = "https://apis.zigbang.com/v2/items/list"
# post 방식일때
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids[:900]
}
response = requests.post(url, params)
response

<Response [200]>

In [35]:
items = response.json()["items"]
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
df.head()

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32819635,전세,16500,0,39.89,서울시 서초구 서초동,7.5
1,32838798,월세,1000,68,37.56,서울시 서초구 서초동,7.5
2,32872608,월세,1000,70,37.56,서울시 서초구 서초동,7.5
3,32838700,전세,50400,0,66.12,서울시 서초구 서초동,9
4,32767020,월세,20000,90,60.12,서울시 서초구 서초동,5


In [27]:
# max row, max column 설정
pd.options.display.max_columns = 50

50
50


In [36]:
# 주소에 역삼동이 있는 데이터만 필터링
result_df = df[df["address1"].str.contains("역삼동")]
result_df = result_df.reset_index(drop=True)
result_df.head()

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32505365,월세,120,110,29.75,서울시 강남구 역삼동,7
1,32828417,월세,120,120,33.06,서울시 강남구 역삼동,15
2,32872905,월세,200,160,31.62,서울시 강남구 역삼동,8
3,32766952,월세,500,50,24.50,서울시 강남구 역삼동,5
4,32836662,월세,500,50,23.14,서울시 강남구 역삼동,5


### 5. 함수 만들기

In [37]:
# 함수 만들기
def oneroom(addr):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]

In [38]:
oneroom("마포구 합정동")

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32864713,전세,9000,0,21.49,서울시 영등포구 당산동4가,13
1,32694522,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
2,32822043,전세,9000,0,16.53,서울시 영등포구 당산동4가,13
3,32827028,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
4,32856780,전세,7000,0,19.83,서울시 영등포구 당산동4가,13
...,...,...,...,...,...,...,...
739,32831032,월세,200,65,18.18,서울시 마포구 성산동,7
740,32775788,전세,16500,0,19.83,서울시 마포구 성산동,1
741,32589981,전세,43000,0,49.00,서울시 마포구 중동,7
742,32862490,전세,43000,0,31.74,서울시 마포구 중동,5


### 6. 모듈 파일 만들기
- .py

In [40]:
%%writefile zigbang.py
# 가장 위의 위치
import requests
import pandas as pd
import geohash2

def oneroom(addr):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]

Writing zigbang.py


In [41]:
import zigbang as zb

In [42]:
df = zb.oneroom("망원동")
df.head()

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32864713,전세,9000,0,21.49,서울시 영등포구 당산동4가,13
1,32694522,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
2,32822043,전세,9000,0,16.53,서울시 영등포구 당산동4가,13
3,32827028,전세,9000,0,19.83,서울시 영등포구 당산동4가,13
4,32856780,전세,7000,0,19.83,서울시 영등포구 당산동4가,13


In [47]:
# 변수목록
%whos

Variable          Type                          Data/Info
---------------------------------------------------------
NamespaceMagics   MetaHasTraits                 <class 'IPython.core.magi<...>mespace.NamespaceMagics'>
get_ipython       function                      <function get_ipython at 0x000001C6F3A43DC8>
getsizeof         builtin_function_or_method    <built-in function getsizeof>
json              module                        <module 'json' from 'C:\\<...>\lib\\json\\__init__.py'>
np                module                        <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
var_dic_list      function                      <function var_dic_list at 0x000001C6FA392678>


In [46]:
# 변수 초기화
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
